# Noughts and Crosses

The goal of this worksheet is for you to implement a simple AI that plays noughts and crosses (also known as tic-tac-toe) using the minimax algorithm. Ultimately we will print out a game where both players play perfectly; it should be a draw.



## Minimax Background

The minimax algorithm is attributed to John von Neumann (1928, `Zur Theorie der Gesellschaftsspiele`), but its key features were described earlier by Émile Borel (1921, `La théorie du jeu et les équations intégrales à noyau symétrique`). It has also been sugested that Charles Babbage may have known about the algorithm. Irregardless of who formalised it, anyone who has ever played Draughts, Chess, Go, or countless other games, has used it. At any point in the game you choose your next move by thinking several ahead, and considering what you and your opponent will do to maximise their respective chances of winning. In the formal version 'several ahead' means all the way to the very end, where one of you has won.

Minimax is an algorithm of game theory (a term invented by von Neumann with Morgenstern), which formalises how to act when you are in competition with others (If you want to know more then `The Joy of Game Theory` by  `Presh Talwalkar` is a friendly introduction). The basic version introduced here is for finding the optimal move in _zero sum_ games that are _turn based_ with _two players_ where everything is _deterministic_ and _observable_, and one has enough computation/memory to exhaust the _search tree_. To define these terms:
 * zero sum: There are n points to be won and they are distributed between the players at the end of the game. In other words, there is no possibility of cooperation, and one players fortune is anther's loss. Any game where one player wins and the other loses satisfies this; a draw is half the points to each player.
 * turn based: Players take turns to make a move.
 * two player: There are two players!
 * deterministic: No dice. The consequence of any action is known, exactly.
 * observable: No secrets, so both players know everything.
 * search tree: This is every possible state the game can be in, and the transitions (moves) between them made by the players.

The minimax algorithm is solving the _Nash equilibrium_ (It's discovery is the subject of the film _A Beautiful Mind_ (2001)) for games with the above properties. Note that many of the above properties can be relaxed with more sophisticated versions. Even when it can't approximations are used: Alpha Go used minimax to beat homosapiens at Go, but never could have held the entire search tree of Go in memory; that would be more states than there are atoms in the universe... even after you bulked up by replacing every atom with a copy of the universe. Instead, it used machine learning to estimate what the probability of winning from any state was, so it didn't have to search to the end.

## Noughts and Crosses
(alternatively, tic-tac-toe)

Two players take turns to place symbols on a 3x3 grid. First one to make a line (horizontal, vertical, or diagonal) of three of their symbol wins. Tradition is that the crosses (`x`) go first, the noughts (`o`) second.

Here is an example game (rendered using ascii art, with only the vertical lines!):
```
 | |       | |      | |x     | |x     |x|x    o|x|x    o|x|x
 |x|  ->   |x|o ->  |x|o ->  |x|o ->  |x|o ->  |x|o ->  |x|o
 | |       | |      | |     o| |     o| |     o| |     o|x|
```
`x` ultimately wins with a vertical line. As for when `o` lost, they did so with their very first move, even if it then took `x` a few moves to force the win.

## Minimax Explanation

Minimax is best understood by starting at the end of a game. Imagine the board looks like this:
```
o|x|x
 |x|o
o|x|
```
The game has ended, and `x` has won. Lets say that the value is `+1` when `x` wins, `-1` when `y` wins, and `0` for a draw. So this state (of the board) has a score of `+1`.

Key to minimax is the realisation that if we can assign a score to the end states then we can assign a score to any other state by assuming the players always play the best move available. For `x` that means the move that maximises the score, for `o` that means the move that minimises the score.

Lets go back a move:
```
o|x|x
 |x|o
o| |
```
How do we assign a score? It is the turn of `x` and they have three choices:
```
o|x|x    o|x|x    o|x|x
x|x|o     |x|o     |x|o
o| |     o|x|     o| |x
```
If we assume the score is defined for each of these states then naturally `x` will play the move associated with ending in the highest scoring state.

We know the score for the middle state is `+1`, as it is a winning state. What about the other two?
The score of the left and right states can be calculated in the exact same way as for this state, the one difference being it will then be the turn of `o`, who will want to minimise the score.

The left state has two choices for `o`:
```
o|x|x    o|x|x
x|x|o    x|x|o
o|o|     o| |o
```
which will be immediately followed by `x` taking the only move it has left and ending the game. On the left this is a draw (score=`0`), and on the right a win for `x` (score=`+1`). `o` wants to minimise so it will choose the left choice - a draw is preferable to a loss. So the left state scores `0` when `x` looks at it, because `x` assumes `o` will do the best they can. The same argument holds for the right choice. So when `x` is taking its move it will always choose the middle option, because it has the highest score, corresponding to victory.

This pattern repeats. Going right back to the start of the game `x` will consider every possible game that can be played, all the way to the end. And by assuming that it always takes the best move, and the opposition always takes the best move avaliable to it (worst move for `x` because it's a zero sum game), it can calculate the score for every state as the minimum or maximum of the next move, depending on whose turn it is. The name of the algorithm, minimax, comes from this repeating pattern of selecting the minimum and maximum score.


## 1. Ascii Art

It can be challenging to develop an algorithm without being able to see what it is doing. The first task is to write a function that prints the state of a board, using the ascii-art style used above.

To represent board states use a tuple of tuples, so that you may index them with `[row][column]`, where `[0][0]` is the top left and `[2][2]` the bottom right. A space (`' '`) indicates an unused slot, a cross (`'x'`) somewhere the first player played, a circle (`'o'`) somewhere the second player played.


In [1]:
def print_state(state):
    """Prints the state."""
    
    for i in range(0,3):                               #Outer loop for rows
        for j in range(0,3):                           #Inner loop for columns
            print(state[i][j], end="", flush=True)     #Prints row on the same line
            if j!=2:                                   #Prevent | from printing after the last element
                print ('|',  end="", flush=True)
              
           
        print ('')                                     #Prints a new line
    pass

        
# Code to test the above; output should be:
#  | | 
#  | | 
#  | | 
#
# o|x|x
#  |x|o
# o|x| 
#
# o|o|o
# x|x|x
#  | | 

print_state(((' ',' ',' '),(' ',' ',' '),(' ',' ',' ')))
print('') # This acts as a newline, so the outputs are not vertically next to each other

print_state((('o','x','x'),(' ','x','o'),('o','x',' ')))
print('')

print_state((('o','o','o'),('x','x','x'),(' ',' ',' ')))

 | | 
 | | 
 | | 

o|x|x
 |x|o
o|x| 

o|o|o
x|x|x
 | | 


## 2. Scoring the End

In order to detect when a winning move has been played, and assign a score to it. We write a function that takes:

Input:
 * A board state, as described above.

Output:
 * If `x` has won return 1.
 * If `y` has won return -1
 * If it is a draw return 0
 * If nobody has won, and moves can still be played, return None.


In [2]:
def score_end(state):
    """Scores the game state if the game has ended, otherwise returns None."""
   
    global a, b, c
    a,b,c = state                                              #Divides state into 3 seperate tuples
    cell1,cell2,cell3 = a                                      #Divides each tuple into 3 seperate elements
    cell4,cell5,cell6 = b
    cell7,cell8,cell9 = c
    
    #x's wins
    
    #horizontal: elements in a single row are same
    if(cell1==cell2==cell3=='x' or cell4==cell5==cell6=='x' or cell7==cell8==cell9=='x'):
        return 1
    #vertical: elements in a single column are same
    if(cell1==cell4==cell7=='x' or cell2==cell5==cell8=='x' or cell3==cell6==cell9=='x'):
        return 1
    #right diagonal: elements in the right diagonal are same
    if(cell1==cell5==cell9=='x'):
        return 1
    #left diagonal: elements in the left diagonal are same
    if(cell3==cell5==cell7=='x'):
        return 1
    
    
    #y's wins
    
    #horizontal
    if(cell1==cell2==cell3=='o' or cell4==cell5==cell6=='o' or cell7==cell8==cell9=='o'):
        return -1
    #vertical
    if(cell1==cell4==cell7=='o' or cell2==cell5==cell8=='o' or cell3==cell6==cell9=='o'):
        return -1
    #right diagonal
    if(cell1==cell5==cell9=='o'):
        return -1
    #left diagonal
    if(cell3==cell5==cell7=='o'):
        return -1
    
    
    #draw
    
    for i in state:                                            #Check each position for a ' '
        if(' ' in i): 
            return None                                        #Game can still be played
    else:
        return 0                                               #Game is a draw
    
    
# Code to test the above function...
print('Expected: None; Obtained:', score_end(((' ',' ',' '),(' ',' ',' '),(' ',' ',' '))))
print('Expected: 1;    Obtained:', score_end((('o',' ','o'),('x','x','x'),(' ',' ',' '))))
print('Expected: 1;    Obtained:', score_end((('o','x','x'),(' ','x','o'),('o','x',' '))))
print('Expected: 1;    Obtained:', score_end((('o','x','x'),(' ','x','o'),('x','o',' '))))
print('Expected: -1;   Obtained:', score_end((('o','x','x'),(' ','x','o'),('o','o','o'))))
print('Expected: -1;   Obtained:', score_end((('o','x',' '),('o','x',' '),('o',' ','x'))))
print('Expected: -1;   Obtained:', score_end(((' ','x','o'),(' ','o','x'),('o',' ','x'))))
print('Expected: 0;    Obtained:', score_end((('o','x','o'),('x','x','o'),('o','o','x'))))

Expected: None; Obtained: None
Expected: 1;    Obtained: 1
Expected: 1;    Obtained: 1
Expected: 1;    Obtained: 1
Expected: -1;   Obtained: -1
Expected: -1;   Obtained: -1
Expected: -1;   Obtained: -1
Expected: 0;    Obtained: 0


## 3. Playing a Move

Given a state, as described above, and a move, e.g. 'place a `'x'` at row 2, column 1' you need a function that will return a __new__ state, with the move having been played.

Input:
 * State before the move.
 * Row to modify.
 * Column to modify.
 * What to place, `'x'` or `'o'`.

Output:
 * A new state, where only the given row/column has been modified.


In [3]:
def play(state, row, col, mark):
    """Returns a new state after the given move has been played."""
    
    a,b,c = state                                                #Divides state into 3 seperate tuples
    
    if(row==0):                                                  #Checks each position and inputs the new move 
        if(col==0):
            a = (mark, state[0][1], state[0][2])
        elif(col==1):
            a = (state[0][0], mark, state[0][2])
        elif(col==2):
            a = (state[0][0], state[0][1], mark)    
        
    elif(row==1):
        if(col==0):
            b = (mark, state[1][1], state[1][2])
        elif(col==1):
            b = (state[1][0], mark, state[1][2])
        elif(col==2):
            b = (state[1][0], state[1][1], mark)
            
    elif(row==2):
        if(col==0):
            c = (mark, state[2][1], state[2][2])
        elif(col==1):
            c = (state[2][0], mark, state[2][2])
        elif(col==2):
            c = (state[2][0], state[2][1], mark)        
        
    return a,b,c                                                #Returns back the new state represented as (a,b,c)
    pass
    

# Code to test the above function - it should play the game given in the 'noughts and crosses' section above...
start = ((' ',' ',' '),(' ',' ',' '),(' ',' ',' '))
move1 = play(start, 1, 1, 'x')
move2 = play(move1, 1, 2, 'o')
move3 = play(move2, 0, 2, 'x')
move4 = play(move3, 2, 0, 'o')
move5 = play(move4, 0, 1, 'x')
move6 = play(move5, 0, 0, 'o')
move7 = play(move6, 2, 1, 'x')

print_state(start)
print('')
print_state(move1)
print('')
print_state(move2)
print('')
print_state(move3)
print('')
print_state(move4)
print('')
print_state(move5)
print('')
print_state(move6)
print('')
print_state(move7)


# The assert statement throws an error if what it is testing is not true.
# The below assert statements confirm that the output of play is nested tuples.
# 'is' when applied to core types (such as tuple) ensures exact equivalence.
assert(type(move7) is tuple)
assert(type(move7[0]) is tuple)
assert(type(move7[1]) is tuple)
assert(type(move7[2]) is tuple)

 | | 
 | | 
 | | 

 | | 
 |x| 
 | | 

 | | 
 |x|o
 | | 

 | |x
 |x|o
 | | 

 | |x
 |x|o
o| | 

 |x|x
 |x|o
o| | 

o|x|x
 |x|o
o| | 

o|x|x
 |x|o
o|x| 


## 4. Finding Possible moves

Given a state we need to know all of the possible moves that the current player can make, that is, a list containing the coordinates of all empty places.

Input:
 * The state, as described above.
 
Output:
 * A list, containg a tuple for every possible move, where each tuple is `(row, column)`. For instance, if the current state is `(('o',' ','o'),('x','x','o'),(' ','o','x'))` the output should be `[(0,1), (2,0)]` noting that the order within the list doesn't matter.


In [4]:

def moves(state):
    """Returns the list of moves that are avaliable from the current state."""
    
    possible_moves = []                                                #Initialise the list of possible moves
    
    for r in range(0,3):
        for c in range(0,3):
            if(state[r][c]==' '):                                      #Checks all empty positions
                position = (r,c)          
                possible_moves.append(position)                        #Adds empty position to possible moves list
            
    return possible_moves
    pass

# Testing the above function. Note that a set is another Python data type;
# it's value here is it doesn't care about order when making comparisons, so if you
# output the list of moves in a different order it still does the right thing...
moves1 = moves(((' ',' ',' '),(' ',' ',' '),(' ',' ',' ')))
assert(set(moves1)==set([(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]))

moves2 = moves((('o',' ','o'),('x','x','x'),(' ',' ',' ')))
assert(set(moves2)==set([(0, 1), (2, 0), (2, 1), (2, 2)]))

moves3 = moves((('o','x','o'),('x','x','o'),('o','o','x')))
assert(moves3==[])

## 5. Minimax

Now to implement the minimax algorithm.

Input:
 * A game state.
 * Whose turn it is, either 'x' or 'o'.

Output:
 * The estimated score of the game.
 * None if the game has ended, or a tuple of (row, column) 


As this exercise is not about understanding the algorithm but practising programming here is psuedocode:
* Use `score_end()` to check if the game has ended, and return the score if so (plus None to indicate the game has ended).
* Use `moves()` to get all possible moves.
* Loop the moves:
  * Use `play()` to imagine the state after the move.
  * Call the minimax `score()` function on each of these moves (recursion) to find out how good they are. Remember it should be the other players turn.
* Select the move that is best for the current player (highest for 'x', lowest for 'o') and return that moves score and `(row, column)`.


In [5]:
#This minimax function is without the cache function

def score(state, player):

    if(player == 'x'):
        best_score = -2                                         #Initialize best score according to the minimizer 
    elif(player == 'o'):                                        #or maximizer player
        best_score = 2
        
    if(score_end(state) == 1 or score_end(state) == -1 or score_end(state) == 0): #Checks whether game has ended
        return score_end(state), None                           #Returns the final score and none as next player
    
    elif(score_end(state) == None):                             #Game has not ended
        player_moves = moves(state)                             #Collectively sends all possible moves 
            
        for m in player_moves:                                  #Loops through all possible moves
            new_state = play(state, m[0], m[1], player)         #Plays a move
            
            if(player == 'x'):                                 
                new_state_score,_ = score(new_state, 'o')       #Recieves score of the new state and player is switched
                
            elif(player == 'o'):
                new_state_score,_ = score(new_state, 'x')
                            
            
            if (player == 'x' and (best_score < new_state_score)):#Compare previous best score to new score 
                best_score = new_state_score                    #If best score is less than new score 
                best_move = m                                   #the values are replaced
                
            elif (player == 'o' and (best_score > new_state_score)):
                best_score = new_state_score                   #If best score is less than new score 
                best_move = m                                  #the values are replaced
                 
                    
        return best_score, best_move    
        
    pass



# Code to test the above...
print('Expected =  0; Obtained:', score(((' ',' ',' '),(' ',' ',' '),(' ',' ',' ')), 'x')[0])
print('Expected =  1; Obtained:', score(((' ',' ',' '),(' ','x','o'),(' ',' ',' ')), 'x')[0])
print('Expected =  1; Obtained:', score((('x',' ',' '),(' ','x','o'),(' ',' ',' ')), 'o')[0])
print('Expected = -1; Obtained:', score((('o',' ',' '),('x',' ',' '),('o',' ',' ')), 'o')[0])



Expected =  0; Obtained: 0
Expected =  1; Obtained: 1
Expected =  1; Obtained: 1
Expected = -1; Obtained: -1


In [6]:
#This minimax function is with the cache function
cache = dict()                                             #Define cache as list

def score(state, player):
    
    if (state, player) in cache:                          #Check if cache is available
        return cache[(state, player)]                     #Return chache
    
    if(player == 'x'):
        best_score = -2
    elif(player == 'o'):
        best_score = 2
        
    if(score_end(state) == 1 or score_end(state) == -1 or score_end(state) == 0):
        return score_end(state), None
    
    elif(score_end(state) == None):
        player_moves = moves(state)
            
        for m in player_moves:
            new_state = play(state, m[0], m[1], player)
            
            if(player == 'x'):
                new_state_score,_ = score(new_state, 'o')
                
            elif(player == 'o'):
                new_state_score,_ = score(new_state, 'x')
            
            
            if (player == 'x' and (best_score < new_state_score)):
                best_score = new_state_score
                best_move = m 
                
            elif (player == 'o' and (best_score > new_state_score)):
                best_score = new_state_score
                best_move = m  
                

        cache[(state,player)] = best_score, best_move      #Utilize cache to save computation power and time 
        return best_score, best_move    
        
    pass



# Code to test the above...
print('Expected =  0; Obtained:', score(((' ',' ',' '),(' ',' ',' '),(' ',' ',' ')), 'x')[0])
print('Expected =  1; Obtained:', score(((' ',' ',' '),(' ','x','o'),(' ',' ',' ')), 'x')[0])
print('Expected =  1; Obtained:', score((('x',' ',' '),(' ','x','o'),(' ',' ',' ')), 'o')[0])
print('Expected = -1; Obtained:', score((('o',' ',' '),('x',' ',' '),('o',' ',' ')), 'o')[0])



Expected =  0; Obtained: 0
Expected =  1; Obtained: 1
Expected =  1; Obtained: 1
Expected = -1; Obtained: -1


## 6. A Perfect Game

Use the above functions to print out a 'perfect' game, using your minimax to select the move each time for both players (computer vs computer). The result should be a draw.


In [7]:

state = ((' ',' ',' '),(' ',' ',' '),(' ',' ',' '))                    #Initial state
player = 'x'                                                           #x starts the game

while(score_end(state) == None):                                       #While there is no end score
    
    if(player == 'x'):
        _,best_move = score(state, 'x')                                #Calculates score of current move
        state = play(state, best_move[0], best_move[1], 'x')           #Plays the move and returns new state
        player = 'o'                                                   #Switches the player
    elif(player == 'o'):    
        _,best_move = score(state, 'o')
        state = play(state, best_move[0], best_move[1], 'o')
        player = 'x'
        
    print_state(state)                                                 #Prints each state
    print()    
            
    
if(score_end(state) == 0):                                             #Verifies whether the game is a draw
    print('Draw')
elif(score_end(state) == 1): 
    print('x wins')
elif(score_end(state) == -1): 
    print('o wins')    

x| | 
 | | 
 | | 

x| | 
 |o| 
 | | 

x|x| 
 |o| 
 | | 

x|x|o
 |o| 
 | | 

x|x|o
 |o| 
x| | 

x|x|o
o|o| 
x| | 

x|x|o
o|o|x
x| | 

x|x|o
o|o|x
x|o| 

x|x|o
o|o|x
x|o|x

Draw


## 7. Slow Player

The above code is slow. It might be your computer is so fast you don't notice the delay, but even so the above is really inefficient. This is because it is repeating calculations - when you call `score()` the result is the same every time for the same parameters. And when you call `score()` for the first time because of the recursion it procedes to call it for all possible parameters. If we cached the results of these computations everything would be much faster for all future calls, at the expense of consuming memory.



In [8]:
import time

cache = dict() # Reset cache otherwise the test would be unfair.

start = time.time()
for _ in range(10):
    score(((' ',' ',' '),(' ',' ',' '),(' ',' ',' ')), 'x')
end = time.time()

print('Time to execute 10 times = %.2f seconds' % (end - start))

Time to execute 10 times = 0.05 seconds
